# Metabolic Network Reconstruction
### Conservation of hierarchical structuring and multi-source compatibility

Updated May 12<sup>th</sup>, 2021 <br>
<i>Jessica Hoban, The Russell Lab at Drexel University</i>


---
## Contents: <br>

>**[Purpose and Overview](#Overview)** <br><br>
>**[Input](#Input)** <br><br>
>**[Setup](#Setup)** <br>

> **[PART A: Genome Annotation](#PartA)**<br><br>
>> **[Step 1: Building the Directory](#Step1)**<br><br>
> **[Step 2: If your genome is in KEGG](#Step2_KEGG)**<br><br>
> **[Step 2: If your genome is in IMG](#Step2_IMG)**<br><br>
> **[Step 2: If your genome is in NCBI](#Step2_NCBI)**<br><br>
> **[Full Pipeline for Part A](#Full_Pipeline_PartA)**<br>

>**[PART B: Network Reconstruction](#PartB)**<br><br>
>> **[Step 3: Filtering and Extracting Pathway Data](#Step3)**<br><br>
> **[Step 4: Determing Seeds/Nonseeds and Children/Parents](#Step4)**<br><br>
> **[Step 5: Send to Files](#Step5)**<br><br>
> **[Step 6: Visualize Network](#Step6)**<br><br>
> **[Step 7: Summary Statistics and Network Comparisons](#Step7)**<br><br>
> **[Full Pipeline for Part B](#Full_Pipeline_PartB)**<br>

---

<br><br>


<br><br><br><br><br><br><br><br><br>

<a id='Overview'></a>
## <span style="margin:auto;display:table">Purpose and Overview</span>

---

### Purpose

This notebook is an end-to-end pipeline for reconstructing metabolic networks, designed to be run in conjunction with the **NetInteract** tool. It is compatible with genomes that have annotations in either the Kyoto Encyclopedia of Genes and Genomes (**KEGG**) <sup>1</sup>, the Joint Genome Institute's Integrated Microbial Genomes system (**IMG**) <sup>2</sup>, or the National Center for Biotechnology Information (**NCBI**) <sup>3</sup>, as well as user-supplied genome annotations.
    
Standard representation for metabolic networks is edge-notation, with substrates and products separated by a tab. However, this formatting does not maintain the <i>origin</i> of those reactions. To understand which overall categories of metabolism (e.g. carbohydrates, amino acids, lipids, etc.) or specific pathways these reactions originated from, the data must be structured hierarchically. Not only can edge-notation files still be extracted in this structure—they can be filtered.

The end data structure is a JSON file that maintains the hierarchical structure of category → pathway → reaction → compound, with additional distinctions between exogenous metabolites ("seeds") and endogenous metabolites ("non-seeds") as defined by the Borenstein lab<sup>4,5</sup>. For optimal user exploration, we also print out CSV files that can be easily browsed.

<br>

### Overview of Process

There are 5 required and several optional steps in this notebook.

> **PART A: Genome Annotation** <br>
>> **1)** Build a custom directory to hold data files. <br><br>
> **2)** Retrieve all KEGG Orthology numbers (representing organismal genes) for each genome and send to file. There is a distinct implementation for each data source, though the outcome is the same. For more details, see each data source's corresponding section. <br><br>
> <span style="margin:auto;display:table;color:blue;"><i>User can then modify the genome annotation results from Part A.</i></span>

> **PART B: Network Reconstruction** <br>
>> **3)** For each pathway, connect to the KEGG API and retrieve the KGML file. Filter this file by the organism's KO numbers. Retrieve all matching reactions and their compounds. <br><br>
> **4)** Determine "seed" (required) and "non-seed" (non-required) compounds using Kosaraju's algorithm. (<i>Theory borrowed from the Borenstein lab<sup>4,5</sup>. Code inspired by the tool PhyloMint <sup>6</sup>).</i> Additionally, determine predecessor/successor relationships between metabolites. <br><br>
> **5)** Send to files (JSON and CSV).<br><br>
> **6)** Visualize each metabolic network and send image to PDF file.<br><br>
> **7)** Compute summary statistics for each network as well as pair-wise comparisons between all networks.

---

<b>References / Additional Resources:</b><br>
><sup>1</sup> https://www.kegg.jp/ <br>
<sup>2</sup> https://img.jgi.doe.gov/cgi-bin/mer/main.cgi <br>
<sup>3</sup> https://www.ncbi.nlm.nih.gov/ <br>
<sup>4</sup> https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-015-0588-y <br>
<sup>5</sup> http://depts.washington.edu/elbogs/NetCooperate/NetCooperateWeb.cgi<br>
<sup>6</sup> https://github.com/mgtools/PhyloMint



----

<br><br><br><br><br><br><br><br><br>

<a id='Input'></a>
## <span style="margin:auto;display:table">Input</span>


----


This section will walk you through the required input for this notebook, which is broken down into 4 main steps:
>**[1) Genome Information](#1GI)** <br><br>
>**[2) For any genomes in the IMG database](#2IMG)** <br><br>
>**[3) For any genomes in the NCBI database](#3NCBI)** <br><br>
>**[4) For any user-supplied genomes](#4User)** <br>


<img src="./Images/Input - Input Folder.png" style="width: 50%;">



---
<br>

<a id='1GI'></a>
### 1) Genome Information

>In the **"Input"** folder, you will find a CSV file called **"Genome Information.csv"**. It contains the following columns:
- ```Organism Name``` <br>
- ```Organism Type```<br>
- ```NCBI ID```  <br>
- ```IMG ID```<br>
- ```KEGG ID``` <br>
- ```User-Supplied ID```

> Please fill this file in with your genomes' information. Here is an example file: <br>
<img src="./Images/Input - Genome Information File.png" style="width: 80%;">

>Some tips:
>- Each genome should have its own row. <br><br>
>- For ```Organism Type```, the options are: ```Host```,  ```Obligate Symbiont```, or ```Facultative Symbiont```.<br><br>
>- Only fill in the IDs for the databases your genome is entered in. Leave the rest of the cells empty. <br><i>(For example, one genome may  be in NCBI and IMG, or it may only in IMG, or it may in all databases.)</i>


---

<br>

<a id='2IMG'></a>
### 2) For any genomes in the IMG database

> Obtaining genome annotation from the IMG database requires one input file that must be manually downloaded. To get this file, please follow these steps (outlined in the diagram below): <br><br>
**1)** Go to the genome entry in IMG. <br>
**2)** Scroll down and click on the **"Export Gene Information"** button.  <br>
**3)** In the page that opens, directly above the table will be a **"Downloads"** button.  <br>
**4)** Click the highlighted link to download the file. <br>
**5)** In the **"Input"** folder, you will find another folder called **"IMG Files"**. Place the downloaded file from step 4 in there. <br>
**6)** Repeat steps 1-5 for all genomes in the IMG database. <br>

<img src="Images/Input - IMG Files.svg" style="width: 50%;">

---

<br>

<a id='3NCBI'></a>
### 3) For any genomes in the NCBI database

> Obtaining genome annotation from the NCBI database requires several manual steps. This notebook relies on a specific form of genome annotation in which each gene is assigned to a KEGG Orthology number <sup>6</sup> (a.k.a. **KO number**). <br><br>
There are no internal links between NCBI genes and KO numbers, (unless the organism is already entered in KEGG, in which case we would simply use the KEGG implementation). Therefore, we need to use a tool called BlastKOALA <sup>7</sup> to do this KO assignment.<br><br>
To do this, please follow these steps (also outlined in diagrams below): <br>

>**1) Obtaining the organism's entire amino acid sequence in FASTA format:**<br>
>>a) Go to the NCBI Assembly page for your genome.<br>
b) On the right-hand side, there is a link for the **"FTP directory for RefSeq assembly"**. Click this link.<br>
c) There will be a directory of links in the page that opens. Select the one that ends in <u><b><span style="color:blue;">protein.faa.gz</span></b></u>; the file will automatically download.<br>
<img src="Images/Input - NCBI Files 1.svg" style="width: 85%;">

>**2) Submitting a BlastKOALA job:**<br>
>>a) Go to https://www.kegg.jp/blastkoala/. <br>
b) Under **"Upload query amino acid sequences in FASTA format"**, select **"Choose File"** and upload your FASTA file from step 1c. <br>
c) Choose the taxonomy group and which database to search based on your organism's taxonomic group. <br>
d) Enter your email address and select **"Request for email confirmation"**.<br>
e) You will get an email asking to confirm submission. Select the **"(Submit)"** link.<br>

>**3) Downloading the BlastKOALA results:**<br>
>>a) Once your BlastKOALA job was completed, you will get an email. Open the link in the email.<br>
b) Click on the **"Download"** button to download the results.<br>
c) Rename the downloaded text file as your genome's NCBI ID (e.g., "ASM960v1.txt").<br>
d) In the **"Input"** folder, you will find another folder called **"NCBI Files"**. Place the downloaded file from the previous step (c) in there. <br>
e) Repeat steps 1-3 for all genomes in the NCBI database. <br>
<img src="Images/Input - NCBI Files 2.svg" style="width: 70%;">

---

<br>

<a id='4User'></a>
### 4) For any user-supplied genomes

> If you have your own genome annotation that you would like to use, please create a CSV file that meets the following criteria:
- Name the file with a unique ID for your genome.<br>
- Have only one column in this CSV file named **"KO Numbers"**. Fill this column with all of your organism's KO numbers.<br>
- In the **"Input"** folder, you will find another folder called **"User-Supplied Files"**. Place this file in there.<br>
<img src="Images/Input - User-Supplied Files 1.svg" style="width: 45%;">

> Here is an example file:<br>
<img src="Images/Input - User-Supplied Files 2.png" style="width: 15%;">

---

<b>References / Additional Resources:</b><br>
><sup>6</sup> https://www.genome.jp/kegg/ko.html <br>
><sup>7</sup> https://www.kegg.jp/blastkoala/ <br>

---

<br><br><br><br><br><br><br><br><br><br>
# <span style="margin:auto;display:table;">Thank you! We'll take it from here.</span>
<br><br><br><br><br>

<br><br><br><br><br><br><br><br><br>

<a id='Setup'></a>
## <span style="margin:auto;display:table">Setup</span>

----


#### Install libraries

To install the BIO.KEGG libraries, run the following:

In [1]:
#conda install -c conda-forge biopython

All other libraries should come pre-installed with Anaconda. If not, Google ```conda install [library_name]``` and there will be an anaconda.org link with the proper syntax.

In [2]:
#conda install -c conda-forge regex 
#conda install -c anaconda urllib3
#conda install -c anaconda requests

#### Import libraries

In [3]:
import re #Regex string matching
import json #JSON serialization & de-serialization
import urllib.parse #URL modification
import requests #Making HTTP requests
from bs4 import BeautifulSoup #Parsing HTML
import pandas as pd #Manipulating tabular data
pd.options.mode.chained_assignment = None #Gets rid of warning
from pprint import pprint # Displays data nicely
from Bio.KEGG import REST #KEGG's REST API
from Bio.KEGG import KGML #KEGG's KGML API
from Bio.KEGG.KGML.KGML_parser import read as kgml_read #Reads KGML pathway map
import os #Traverse directory
import glob #Grab files that match pattern in directory
import math #For determining NaN values
from shutil import copyfile #For copying files over
import itertools # Creates permutations of organism pairs
import networkx as nx # Network visualization
import matplotlib.pyplot as plt # Plotting
import seaborn as sns #Heatmaps

#### Define auxilary functions

In [4]:
#---------------------------------------------------------
# Extract the network in tab-delimited edge-notation format
def extract_TSV(org_dict):

    # Initialize empty list to hold edges
    edges = []

    # Iterate over categories, pathways, and reactions
    for cat_dict in org_dict["organism"]["categories"]:
        for path_dict in cat_dict["pathways"]:
            for reaction_dict in path_dict["reactions"]:
                for e in reaction_dict["edges"]:
                    if e not in edges:
                        edges.append(e) # Append edges to list

    # Return list of tab-delimited edges (e.g., ["C#####\tC#####", ...])
    return edges

#---------------------------------------------------------

<br><br><br><br><br><br><br><br><br><br>


<a id='PartA'></a>
# <span style="margin:auto;display:table;color:blue;">PART A: Genome Annotation</span>

<br><br><br><br><br><br><br><br><br>

<a id='Step1'></a>
## <span style="margin:auto;display:table">Step 1: Building the Directory</span>

----


#### Required input: 
> ```genome_information_filepath```  &emsp; *(default = "./Input/Genome Information.csv")*<br>

#### Overview of process: <br>

This notebook was designed to work in conjunction with a specific directory structure (see diagram below).

<img src="./Images/Directory Structure.svg" style="width: 60%;">

The following code 1) creates this directory structure based on the **"Genome Information.csv"** file that the user filled in during the Input section, and 2) returns input that will be used in subsequent steps (e.g., genome information, filepaths for intermediate and final data files).

In [5]:
"""
Steps:
1) Read in the CSV file of genome information as a pandas DataFrame
2) For each genome:
    - Create the appropriate folders (as designated in the structural diagram above)
    - Return required input for future steps (genome information as well as filepaths)
3) Populate and return 3 lists of dictionaries

Returns: 

    Three lists of dictionaries, 
    one for each organism type (host, obligate symbiont, facultative symbiont)

    Each list of dictionaries is in the form:
    
    [
        {
        "database" : 
        "organism_name" : 
        "organism_ID" : 
        "intermediate_filepath" :
        "destination_filepath_json" : 
        "destination_filepath_csv" : 
        "destination_filepath_csv_nonseeds" :
        "destination_filepath_pdf" : 
        "input_filepath" : 
        "partA_annotation_complete" :
        "partB_network_complete" :
        }
    ]

    Note: "input_filepath" will only be returned for genomes in IMG and NCBI

"""

def step1(genome_information_filepath = "./Input/Genome Information.csv"):
    
    # Read in "Genome Information.csv" as a pandas DataFrame
    genome_information = pd.read_csv(genome_information_filepath)

    # Intialize lists to hold input information
    all_inputs = []
    
    # Iterate over rows in DataFrame
    for _, row in genome_information.iterrows():

        # Iterate over database columns
        for database in ["KEGG ID", "IMG ID", "NCBI ID","User-Supplied ID"]:

            # Check that the cell isn't NaN
            if not (isinstance(row[database], float) and  math.isnan(row[database])):

                # Convert IMG IDs to integers(otherwise a ".0" will be added to the end)
                if database == "IMG ID":
                    row[database] = int(row[database])
                    
                # Grab information into new variables (easily readability)
                organism_ID = str(row[database])
                organism_name = row['Organism Name']
                organism_type = row['Organism Type']
                database_name = database[:-3]
                    
                # Define the genome's folderpath
                folderpath = "./Metabolic Networks/" + organism_type + "/" + organism_name + "/" + database_name + " (" + organism_ID + ")"
                # Define the names of the lowest-level folders 
                folder_names = [folderpath + "/Genome Annotation", folderpath + "/Metabolic Network"]
                
                # Iterate over folder names
                for folder_name in folder_names:
                    # Recursively make this directory, creating any intermediate directories as needed
                    try:
                        os.makedirs(folder_name)
                    # Don't make anything if the directory already exists
                    except FileExistsError:
                        pass
                  
                # Define where we want future intermediate files to go (i.e., in the "Genome Annotation" folder for that genome)
                intermediate_filepath = folder_names[0] + "/" + database_name + "_" + organism_ID + "_KO_numbers.csv"
                
                # Define where we want future destination files to go (i.e., in the "Metabolic Network" folder for that genome)
                destination_filepath_json = folder_names[1] + "/" + database_name + "_" + organism_ID + "_metabolic_network.json"
                destination_filepath_csv = folder_names[1] + "/" + database_name + "_" + organism_ID + "_metabolic_network.csv"
                destination_filepath_csv_nonseeds = folder_names[1] + "/" + database_name + "_" + organism_ID + "_metabolic_network_nonseeds.csv"
                destination_filepath_pdf = folder_names[1] + "/" + database_name + "_" + organism_ID + "_metabolic_network.pdf"
                
                # Aggregate all of this input into one dictionary
                genome_input = {"database" : database_name,\
                                "organism_name" : organism_name,\
                                "organism_ID" : organism_ID,\
                                "organism_type" : organism_type,\
                                "destination_filepath_json" : destination_filepath_json,\
                                "destination_filepath_csv" : destination_filepath_csv,\
                                "destination_filepath_csv_nonseeds" : destination_filepath_csv_nonseeds,\
                                "destination_filepath_pdf" : destination_filepath_pdf,\
                                "intermediate_filepath": intermediate_filepath,\
                                "partA_annotation_complete": False,\
                                "partB_network_complete": False}
                    
                #------------------------
                # Initialize an indicator variable which will show if the required input files have been supplied
                complete_input = True
                
                # All database sources besides KEGG (i.e., IMG, NCBI, User-supplied) require input files.
                # The following section finds those files and copies them into an appropriate location
                # in the directory structure.
                if database_name != "KEGG":
                    
                    # Find input file for that genome (based on database and ID)
                    # For example, an IMG genome should have a corresponding input file in the folder "IMG Files"
                    input_filepath_source = glob.glob("./Input/" + database_name + " Files/" + "*" + organism_ID + "*")
                    
                    # If the user didn't input the file properly (i.e., nothing found)
                    if input_filepath_source == []:
                        complete_input = False # Update indicator variable
                        print("Error: No " + database_name + " File detected for genome ID " + organism_ID)
                        
                    # If input file was found
                    else:
                        
                        # For IMG and NCBI, these require an "input file" which will be used to generate an 
                        # "intermediate file" after Step 2
                        if database_name in ["IMG", "NCBI"]:
                        
                            # Define where we want to copy it into
                            input_filepath_destination = folder_names[0] + "/" + input_filepath_source[0].split("/")[-1]
                            # Add to input dictionary
                            genome_input["input_filepath"] = input_filepath_destination
                            # Copy from "./Input/[Database] Files" into appropriate genome's "./Genome Annotation" folder
                            try:
                                copyfile(input_filepath_source[0], input_filepath_destination)
                            except FileExistsError:
                                print("Input file already in correct location.")
                        
                        # For user-supplied genomes, these annotations have already been completed, so they don't need
                        # to pass through Step 2. Therefore, we just use this "input file" as the "intermediate file"
                        elif database_name == "User-Supplied":
                            
                            # Copy from "./Input/[Database] Files" into appropriate genome's "./Genome Annotation" folder
                            try:
                                copyfile(input_filepath_source[0], genome_input["intermediate_filepath"])  
                            except FileExistsError:
                                print("Input file already in correct location.")
                
                #------------------------
                # Check that this genome hasn't been ran before (check part A & part B separately)
                
                # Check for complete genome annotations from Part A
                if os.path.isfile(genome_input["intermediate_filepath"]) == True:
                    genome_input["partA_annotation_complete"] = True
                
                # Check for complete genome annotations from Part B
                if os.path.isfile(genome_input["destination_filepath_json"]) == True:
                    genome_input["partB_network_complete"] = True
                
                #------------------------
                # Make sure required input files are present
                if complete_input == True:
                    
                    # Append to list of all genome inputs
                    all_inputs.append(genome_input) 
                    
    # Return
    return all_inputs
                     

<br><br><br><br><br><br><br><br><br>

<a id='Step2_KEGG'></a>
## <span style="margin:auto;display:table">Step 2: If your genome is in KEGG</span>
#### Required input: 
> ```genome_input``` <br>
> ```all_KO_names``` <br>
> ```all_KO_descriptions```

#### Overview of process: <br>
The Kyoto Encyclopedia of Genes and Genomes (hereafter called KEGG) maintains an API for public use<sup>1</sup>. We will be accessing that API using the Biopython collection that we installed in the set-up section. In Biopython there is a package called ```Bio.KEGG```; we will use several of its modules in this notebook, but for this step we will only be using the ```REST``` module<sup>2</sup>. 

First, we use ```REST.kegg_list``` to obtain all available pathways for the inputted organism. Then, we use ```REST.kegg_get``` to obtain the pathway entry file for each pathway, which is returned as plain text<sup>3</sup>. There are two main sections of interest in this file: <b>CLASS</b> and <b>GENE</b>. We check that the first part of the <b>CLASS</b> section is "Metabolism" (since we are only concerned with metabolic pathways). Then, for each line in the <b>GENE</b> section, we use regular expressions to extract the gene's KO number. We use the supplied dictionaries, ```all_KO_names``` and ```all_KO_descriptions```, to map relevant information to each gene based on its KO number.


This data is printed to a CSV file in the <b>Genome Annotation</b> folder for this genome, using the ```intermediate_filepath``` that was automatically determined in the set-up section. This allows for optional user curation of the KO numbers before proceeding to Step 2.


#### References / Additional Resources: <br>
> <sup>1</sup> https://www.kegg.jp/kegg/rest/keggapi.html <br>
> <sup>2</sup> https://biopython.org/DIST/docs/api/Bio.KEGG.REST-module.html <br>
> <sup>3</sup> Chapter 18.2 in http://biopython.org/DIST/docs/tutorial/Tutorial.html 


----

In [6]:
"""
Steps:
1) Use REST.kegg_list to obtain all available pathways for the organism
2) Use REST.kegg_get to obtain the pathway entry file for each pathway
2) Check that the first part of the CLASS section is "Metabolism" 
   & extract the second part as categoryName
3) Use regular expressions to extract the KO number(s) for that pathway
4) Print to a CSV file in the "Genome Annotation" folder with the following columns:
    - "Database"
    - "Database Gene ID"
    - "KO Number"
    - "KO Name"
    - "KO Description"

*Note*: Does not return anything, because we "break" the pipeline into 2 steps
to allow for manual user curation.

"""

def step2_KEGG(genome_input, all_KO_names, all_KO_descriptions):
    
    # Extract organism_ID from the genome_input we found in the setup step
    org_ID = genome_input["organism_ID"]

    # Initialize DataFrame
    # (will be populated with 1 row per gene)
    org_genes_df = pd.DataFrame()

    # Find length of organism ID -- will be used in string matching
    org_ID_len = len(org_ID)

    #------------GET ALL PATHWAYS FOR ORGANISM------------

    # Get all organism pathways (downloaded as plain text)
    pathways_str = REST.kegg_list("pathway", org_ID).read()
    
    # Convert plain text to set of pathway IDs
    pathways_set = set()
    for line in pathways_str.rstrip().split("\n"):
        entry, _ = line.split("\t")
        pathways_set.add(entry)

    # Convert set to list (set -> list == unique entries only)
    pathways = list(pathways_set)

    # How the data looks at this stage:
    # ['path:org_ID#####', 'path:org_ID#####', 'path:org_ID#####'... ]

    #------------FILTER FOR ONLY METABOLIC PATHWAYS------------
    
    # Iterate over all pathways
    for path in pathways:
        
        # Filter out pathways that start with org_ID011, org_ID012, org_ID010, org_ID07
        # (i.e. global, overview, or structural maps without KGML files)
        if not path.startswith(("011", "012", "010", "07"), org_ID_len + 5):
            
            # Access KEGG API to get pathway entry file
            # (same as https://www.kegg.jp/dbget-bin/www_bget?pathway+org_ID#####)
            pathway_file = REST.kegg_get(path).read()

            # Extract pathway ID only (i.e. "path:org_ID#####" -> "#####")
            path_index = 5 + org_ID_len
            path_ID = path[path_index:]      

            # Initialize variable to keep track of file section
            current_section = None

            # Split the pathway file by newline characters 
            # For each line, strip any trailing characters to the right of the line
            for line in pathway_file.rstrip().split("\n"):

                # Section names are within first 12 characters
                section = line[:12].strip()

                if not section == "":
                    # Keep track of the section
                    current_section = section

                if current_section == "CLASS":
                    # Category_class will show if it's a metabolic pathway or not
                    category_class, category = line[12:].split("; ")

                if (current_section == "GENE"):

                    # Find KO section (in case there's another unrelated KO in the description)
                    # Should look like [KO:K#####] or [KO:K##### K#####] 
                    KO_section_regex = re.compile(r"\[KO:(K\d{5})+( *(K\d{5}))*]")
                    KO_section_search = KO_section_regex.search(line)

                    if KO_section_search != None:
                        # Within this KO section, grab all KO #s
                        start,end = KO_section_search.span()
                        KO = re.findall(r"K\d{5}", line[start:end])[0]

                        # Grab gene ID (anything before first space)
                        database_gene_ID = line[12:].split(" ")[0]

                        # Initialize and populate row
                        gene_row = {"Database": "KEGG",\
                                    "Database Gene ID": database_gene_ID,\
                                    "KO Number": KO,\
                                    "KO Name" : all_KO_names[KO],\
                                    "KO Description" : all_KO_descriptions[KO]}

                        # Filter for only "Metabolism" as Class
                        if category_class == "Metabolism":

                            # Append to DataFrame
                            org_genes_df = org_genes_df.append(gene_row, ignore_index = True)
                
    # Print final DataFrame to file
    org_genes_df.to_csv(genome_input["intermediate_filepath"], index = False)

    # Print progress
    print("Step 1 complete.")  
    

<br><br><br><br><br><br><br><br><br>

<a id='Step2_IMG'></a>
## <span style="margin:auto;display:table">Step 2: If your organism is in IMG</span>
#### Required input: 
> ```genome_input``` <br>
> ```all_KO_names``` <br>
> ```all_KO_descriptions```

#### Overview of process: <br>

Unlike KEGG, the Joint Genome Institute's Integrated Microbial Genomes system (hereafter called IMG) does not have an API that allows us to extract KO numbers. Therefore, we will extract this information from the input file the user acquired during the previous [Input](#Input) section. 

We use this **Input File** to return all genes that have KO annotations, as well as relevant information for each gene (e.g., KO number, Locus Tag, IMG gene ID). We also use the supplied dictionaries, ```all_KO_names``` and ```all_KO_descriptions```, to map relevant information to each gene based on its KO number.

This data is printed to a CSV file in the **"Genome Annotation"** folder for this genome, using the ```intermediate_filepath``` that was automatically determined in the set-up section. This allows for optional user curation of the KO numbers before proceeding to Step 2.

<br><br>

----



In [7]:
"""
Steps:
1) Read in the XLS file of genome information as a pandas DataFrame
2) Filter the DataFrame for KO information only
4) Print to a CSV file in the "Genome Annotation" folder with the following columns:
    - "Database"
    - "Database Gene ID"
    - "Locus Tag"
    - "KO Number"
    - "KO Name"
    - "KO Description"
 

*Note*: Does not return anything, because we "break" the pipeline into 2 steps
to allow for manual user curation.

"""

def step2_IMG(genome_input, all_KO_names, all_KO_descriptions):
    
    # Read in as pandas DataFrame
    input_df = pd.read_csv(genome_input["input_filepath"], sep = "\t", header = 0, dtype = {'gene_oid':'str'})

    # Drop null values (the file is weirdly formatted with blank lines in between)
    input_df.dropna(axis = 0, how = "all", inplace = True)

    # Extract subset of DataFrame with only KO information
    ko_df = input_df[["gene_oid", "Locus Tag","Source"]].loc[input_df['Source'].str.contains("KO:")]

    # Add database column
    ko_df["Database"] = "IMG"

    # Rename columns
    ko_df = ko_df.rename(columns={"gene_oid":"Database Gene ID",\
                                 "Source":"KO Number"})

    # Remove "KO:" from "KO Number" column
    ko_df["KO Number"] = ko_df["KO Number"].map(lambda x: x[3:])
    
    # Use supplied dictionaries to map KO numbers -> KO names and descriptions
    ko_df["KO Name"] = ko_df["KO Number"].apply(lambda x: all_KO_names.get(x))
    ko_df["KO Description"] = ko_df["KO Number"].apply(lambda x: all_KO_descriptions.get(x))
            
    # Print final DataFrame to file
    ko_df.to_csv(genome_input["intermediate_filepath"], index = False)

    # Print progress
    print("Step 1 complete.")   

<br><br><br><br><br><br><br><br><br>

<a id='Step2_NCBI'></a>
## <span style="margin:auto;display:table">Step 2: If your organism is in NCBI</span>
#### Required input: 
> ```genome_input``` <br>
> ```all_KO_names``` <br>
> ```all_KO_descriptions```

#### Overview of process: <br>

Unlike KEGG, the National Center for Biotechnology Information (hereafter called NCBI) does not have an API that allows us to extract KO numbers. Therefore, we will extract this information from the input file the user acquired during the previous [Input](#Input) section. 

We use this **Input File** to return all genes that have KO annotations, as well as relevant information for each gene (e.g., KO number and Locus Tag). We also use the supplied dictionaries, ```all_KO_names``` and ```all_KO_descriptions```, to map relevant information to each gene based on its KO number.

This data is printed to a CSV file in the **"Genome Annotation"** folder for this genome, using the ```intermediate_filepath``` that was automatically determined in the set-up section. This allows for optional user curation of the KO numbers before proceeding to Step 2.

-----


In [8]:
"""
Steps:
1) Read in the text file of genome information as a pandas DataFrame
2) Add column names
4) Print to a CSV file in the "Genome Annotation" folder with the following columns:
    - "Database"
    - "Locus Tag"
    - "KO Number"
    - "KO Name"
    - "KO Description"
    
*Note*: Does not return anything, because we "break" the pipeline into 2 steps
to allow for manual user curation.

"""

def step2_NCBI(genome_input, all_KO_names, all_KO_descriptions):
    
    # Read in as pandas DataFrame
    input_df = pd.read_csv(genome_input["input_filepath"], sep = "\t", \
                            header = None, names = ["Locus Tag", "KO Number"])

    # Add database column
    input_df["Database"] = "NCBI"

    # Use supplied dictionaries to map KO numbers -> KO names and descriptions
    input_df["KO Name"] = input_df["KO Number"].apply(lambda x: all_KO_names.get(x))
    input_df["KO Description"] = input_df["KO Number"].apply(lambda x: all_KO_descriptions.get(x))
          
    # Remove any rows without KO #s
    input_df = input_df[input_df["KO Number"].notna()]
    
    # Print final DataFrame to file
    input_df.to_csv(genome_input["intermediate_filepath"], index = False)

    # Print progress
    print("Step 1 complete.")     


<br><br><br><br><br><br><br><br><br>

<a id='Full_Pipeline_PartA'></a>
## <span style="margin:auto;display:table;">Full Pipeline for Part A</span>



In [9]:
# Steps 1-2
def full_pipeline_partA():
    
    # Get genome inputs based on their 
    all_inputs = step1()
    
    #------------GET DICTIONARY OF KO NAMES {ID:Name}------------
    all_KO_names = {}
    all_KO_descriptions = {}

    # Preferred method is getting this information directly from the API
    try:
        orthology_str = REST.kegg_list("orthology").read()
        
    # However, since the API is not always reliable, we've pre-scraped
    # this information into a text file
    except:
        with open(r"./Supplementary Materials/Orthology_List.txt","r") as f:
            orthology_str = str(f.read())

        for line in orthology_str.rstrip().split(r"\n"):
            KO_ID, KO_name_description = line.split(r"\t")

            if r";" in KO_name_description:
                KO_name, KO_description = KO_name_description.split(r";", 1)

            #One KO annotation is in a different format, no semi-colon 
            #("CCAAT/enhancer binding protein (C/EBP), other")
            else: 
                KO_name = KO_name_description
                KO_description = ""

            all_KO_names[KO_ID[3:]] = KO_name
            all_KO_descriptions[KO_ID[3:]] = KO_description.strip()
        
    #------------
    # Iterate over inputs
    for genome_input in all_inputs:
        
        # Print which genome is currently in progress
        print(genome_input["organism_name"], genome_input["database"], genome_input["organism_ID"])
        
        # Check that the annotation isn't already complete
        if genome_input["partA_annotation_complete"] == True:
            print("Part A already complete for this genome.")
        
        else:
            # Execute the correct Step 2 script based on the genome's database
            if genome_input["database"] == "KEGG":
                step2_KEGG(genome_input = genome_input, \
                           all_KO_names = all_KO_names, \
                           all_KO_descriptions = all_KO_descriptions)

            elif genome_input["database"] == "IMG":
                step2_IMG(genome_input = genome_input, \
                           all_KO_names = all_KO_names, \
                           all_KO_descriptions = all_KO_descriptions)

            elif genome_input["database"] == "NCBI":
                step2_NCBI(genome_input = genome_input, \
                           all_KO_names = all_KO_names, \
                           all_KO_descriptions = all_KO_descriptions)
            
    # Print progress
    print("Part A of Full Pipeline Complete")
    
    # Return all_inputs for part 2
    return all_inputs


In [10]:
%time all_inputs = full_pipeline_partA()

Acyrthosiphon pisum KEGG api
Part A already complete for this genome.
Buchnera aphidicola Tuc7 NCBI ASM2106v1
Part A already complete for this genome.
Buchnera aphidicola 5A NCBI ASM2108v1
Part A already complete for this genome.
Buchnera aphidicola APS NCBI ASM960v1
Part A already complete for this genome.
Regiella insecticola LSR1 NCBI ASM14362v1
Part A already complete for this genome.
Rickettsiella viridis Ap-RA04 NCBI ASM396675v1
Part A already complete for this genome.
Serratia symbiotica Tuscon NCBI ASM18648v1
Part A already complete for this genome.
Fukatsuia symbiotica 5D NCBI ASM312242v1
Part A already complete for this genome.
Hamiltonella defensa 5AT (AA) NCBI ASM2170v1
Part A already complete for this genome.
Hamiltonella defensa NY26 (AA) NCBI New_ASM277729v1
Part A already complete for this genome.
Hamiltonella defensa MI47 (BB) NCBI New_ASM226940v1
Part A already complete for this genome.
Hamiltonella defensa 5D (BB) NCBI New_ASM312244v1
Part A already complete for this

<br><br><br><br><br><br><br><br><br><br>

<a id='PartB'></a>
# <span style="margin:auto;display:table;color:blue;">PART B: Network Reconstruction</span>

<br><br><br><br><br><br><br><br><br>

<a id='Step3'></a>
## <span style="margin:auto;display:table">Step 3: Filtering and Extracting Pathway Data</span>


#### Required input: 
>```genome_input``` <br>
>```all_compound_names```

#### Overview of process:
For Step 3, we add in reaction-level and compound-level data to the output from Step 2. To do this, we're going to access KEGG Markup Language (KGML) files <sup>1</sup> from the KEGG API. These XML-style files describe the network graphs that you see on the KEGG website. 
We retrieve these files using ```REST.kegg_get``` and parse them using ```KGML_parser.read```.

It's important to note that there are different prefixes for KEGG pathways <sup>2</sup>. The number 00010 represents "Glycolysis / Gluconeogenesis", but what's in front changes the contents of the KGML file. For example, putting an organism ID in front (i.e. "hde00010") makes it specific to <i>Hamiltonella defensa 5AT</i>, highlighting only the KO numbers that organism has genes for. However, putting a "ko" in front (i.e. "ko00010") makes it non-specific to any organism, and highlights all genes.

By accessing the non-specific "ko" KGML file for each pathway, we can then filter it by the KO numbers we retrieved in Step 1. We retrieve data for each reaction and its compounds. We also include edge-notation format for each reaction, to be used in Step 4. The final dictionary is updated and returned as output.

**A quick note:** Downloading the organism-specific file for an organism already in KEGG gives you the same results as downloading the non-specific file and filtering it by KO number. The reason for breaking up these steps is so that the code is modularized to work with data sources other than KEGG, including IMG and NCBI. 

---

 #### References / Additional Resources: <br>

><sup>1</sup> https://www.kegg.jp/kegg/xml/docs/ <br>
><sup>2</sup> Pathway Identifiers section https://www.kegg.jp/kegg/pathway.html <br>

-----


In [11]:
"""
Steps:
1) For each pathway, retrieve the non-specific KGML file using REST.kegg_get
2) Use the output from Step 2 to find the KO numbers for each pathway,
    filter the KGML file using those
3) Extract reaction and compound data
4) Format reactions into edge-notation
5) Populate and return dictionary

Returns: a dictionary in the form 
    {
        "organism": {
            "organismID" :
            "organismName" :
            "organismSource" :
            "organismType" :
            "categories" : [
                {
                "categoryName" :
                "pathways" : [
                    {
                    "pathwayID" :
                    "pathwayName" :
                    "allKOnumbers" :
                    "reactions" : [
                        {
                        "reactionID" :
                        "reactionType" :
                        "KOnumbers" :
                        "edges" :
                        "compounds" : [
                            {
                            "compoundID" :
                            "compoundType" :
                            "compoundName":
                            }
                            ,...
                        ]
                        }
                        ,...
                    ]
                    }
                    ,...
                ]
                }
                ,...
            ]
        }
        
    }

"""

def step3(genome_input, all_compound_names):
        
    #------------------------
    # Extract edge-notation format given the substrates and products
    # (i.e., "[substrate\tproduct, ...]")
    def edge_notation(sub_lst, prod_lst): #tab-separated

        # Initialize list of edges
        edge_lst = []
        
        # Iterate over substrates
        for sub in sub_lst:
            # Iterate over products
            for prod in prod_lst:

                # Create reaction in edge-notation ("substrate\tproduct")
                line = sub + "\t" + prod
                # Append to edges list
                edge_lst.append(line)

        # Return
        return edge_lst
    
    #------------------------
    # Read in file from Step 2
    intermediate_file = pd.read_csv(genome_input["intermediate_filepath"])
    
    # Grab all of organism's KO numbers from Step 2 file
    org_KO_numbers = intermediate_file["KO Number"].tolist()
    
    # Initialize the main dictionary
    org_dict = {}
    # Establish root element
    org_dict["organism"] = {} 
    # Fill in organism descriptors
    org_dict["organism"]["organismID"] = genome_input["organism_ID"]
    org_dict["organism"]["organismName"] = genome_input["organism_name"]
    org_dict["organism"]["organismSource"] = genome_input["database"]
    org_dict["organism"]["organismType"] = genome_input["organism_type"]
    
    # Explicitly define category-pathway structure for consistency 
    org_dict["organism"]["categories"] = [{"categoryName" : "Carbohydrate metabolism", \
                                               "pathwayIDs" : ["00010", "00020", "00030", "00040", "00051", "00052", "00053", "00500", "00520", "00620", "00630", "00640", "00650", "00660", "00562"]}, \
                                          {"categoryName" : "Energy metabolism", \
                                               "pathwayIDs" :  ["00190", "00195", "00196", "00710", "00720", "00680", "00910", "00920"]}, \
                                          {"categoryName" : "Lipid metabolism", \
                                               "pathwayIDs" :  ["00061", "00062", "00071", "00073", "00100", "00120", "00121", "00140", "00561", "00564", "00565", "00600", "00590", "00591", "00592", "01040"]}, #"00072" was removeds
                                          {"categoryName" : "Nucleotide metabolism", \
                                               "pathwayIDs" :  ["00230", "00240"]}, \
                                          {"categoryName" : "Amino acid metabolism", \
                                               "pathwayIDs" :  ["00250", "00260", "00270", "00280", "00290", "00300", "00310", "00220", "00330", "00340", "00350", "00360", "00380", "00400"]}, \
                                          {"categoryName" : "Metabolism of other amino acids", \
                                               "pathwayIDs" :  ["00410", "00430", "00440", "00450", "00460", "00470", "00480"]}, #"00471","00472", "00473" removed and "00470" added
                                          {"categoryName" : "Glycan biosynthesis and metabolism", \
                                               "pathwayIDs" :  ["00510", "00513", "00512", "00515", "00514", "00532", "00534", "00533", "00531", "00563", "00601", "00603", "00604", "00540", "00542", "00541", "00550", "00511", "00571", "00572"]}, # added "00542"
                                          {"categoryName" : "Metabolism of cofactors and vitamins", \
                                               "pathwayIDs" :  ["00730", "00740", "00750", "00760", "00770", "00780", "00785", "00790", "00670", "00830", "00860", "00130"]}, \
                                          {"categoryName" : "Metabolism of terpenoids and polyketides", \
                                               "pathwayIDs" :  ["00900", "00902", "00909", "00904", "00906", "00905", "00981", "00908", "00903", "00281", "01052", "00522", "01051", "01059", "01056", "01057", "00253", "00523", "01054", "01053", "01055"]}, \
                                          {"categoryName" : "Biosynthesis of other secondary metabolites", \
                                               "pathwayIDs" :  ["00940", "00945", "00941", "00944", "00942", "00943", "00901", "00403", "00950", "00960", "01058", "00232", "00965", "00966", "00402", "00311", "00332", "00261", "00331", "00521", "00524", "00525", "00401", "00404", "00405", "00333", "00254", "00999", "00998", "00997"]}, \
                                          {"categoryName" : "Xenobiotics biodegradation and metabolism", \
                                               "pathwayIDs" :  ["00362", "00627", "00364", "00625", "00361", "00623", "00622", "00633", "00642", "00643", "00791", "00930", "00363", "00621", "00626", "00624", "00365", "00984", "00980", "00982", "00983"]}]

    #------------------------          
    # Iterate over categories
    for cat_dict in org_dict["organism"]["categories"]:
        
        # Initialize list to hold pathway dictionaries
        # (Will be converting the list of pathway IDs into a list of pathway dictionaries)
        cat_dict["pathways"] = []
        
        # Iterate over pathway_IDs
        for path_ID in cat_dict["pathwayIDs"]:
            
            # Initialize pathway dictionary
            path_dict = {}
            path_dict["pathwayID"] = path_ID 
            
            # Initialize sets that will hold all KO numbers and reactions within
            # this pathway that the organism is capable of
            path_dict["allKOnumbers"] = set()
            path_dict["reactions"] = []
            
            # Get KGML file for whole pathway
            KO_path = "ko" + str(path_ID)
            
            #print(KO_path)
            
            KGML_file = REST.kegg_get(KO_path, "kgml").read()
            
            # Extract pathway name from KGML file
            path_dict["pathwayName"] = kgml_read(KGML_file).title

            # Grab reaction_entries from KGML file
            reaction_entries = kgml_read(KGML_file).reaction_entries
            
            # Initialize a dict of node_ids to be filled with any reaction entry
            # that the organism is capable of (i.e., has a KO number for)
            org_node_IDs = {} 

            # Iterate over reaction_entries
            for k in reaction_entries:

                # Grab kos per reaction_entry
                KGML_KOs = [i[3:] for i in re.findall(r"ko:K\d{5}", k.name)]

                # If there are any KO #s in common
                if any(x in KGML_KOs for x in org_KO_numbers):
                    # Grab them
                    both = [KO for KO in KGML_KOs if KO in org_KO_numbers]
                    
                    # Add to dictionary
                    org_node_IDs[k.id] = both

            # Grab all reactions in pathway from KGML file
            reactions = kgml_read(KGML_file).reactions

            # Iterate over reactions
            for k in reactions:
                
                # Grab the ones that match the organism's reaction entry
                if k.id in org_node_IDs.keys():

                    # Grab all substrate entries
                    substrates1 = [s.name for s in k.substrates]
                    # Make sure they fit the right structure (i.e., C##### or G#####)
                    substrates = [i for s in substrates1 for i in re.findall(r'C\d{5}|G\d{5}', s)]
                    
                    # Grab all product entries
                    products1 = [s.name for s in k.products]
                    # Make sure they fit the right structure (i.e., C##### or G#####)
                    products = [i for s in products1 for i in re.findall(r'C\d{5}|G\d{5}', s)]

                    # Convert to uni-directional/ one-way reactions to tab-delimited edge notation
                    if k.type == 'irreversible': 
                        edge_lst = edge_notation(substrates, products)
                        
                    # Convert to bi-directional reactions to tab-delimited edge notation
                    elif k.type == 'reversible': 
                        edge_lst1 = edge_notation(substrates, products)
                        edge_lst2 = edge_notation(products, substrates)
                        edge_lst = edge_lst1 + edge_lst2

                    # Initialize and populate reaction dictionary
                    reaction_dict = {}
                    reaction_dict["reactionID"] = k.id
                    reaction_dict["reactionType"] = k.type 
                    reaction_dict["edges"] = edge_lst
                    reaction_dict["KOnumbers"] = org_node_IDs[k.id]
                    reaction_dict["compounds"] = []
                    
                    # Add reaction KOnumbers to full set of KO numbers for pathway
                    path_dict["allKOnumbers"].update(org_node_IDs[k.id])
                    
                    # Initialize and populate compound dictionaries
                    for sub in substrates:
                        
                        compound_dict = {"compoundID": sub,\
                                        "compoundType": "substrate",\
                                        "compoundName": all_compound_names[sub]}
                        
                        # Add to reaction dictionary
                        reaction_dict["compounds"].append(compound_dict)
                    
                    # Initialize and populate compound dictionaries
                    for prod in products:
                        compound_dict = {"compoundID": prod,\
                                        "compoundType": "product",\
                                        "compoundName": all_compound_names[prod]}
                        
                        # Add to reaction dictionary
                        reaction_dict["compounds"].append(compound_dict)
                    
                    # Add reaction dictionary to pathway dictionary
                    path_dict["reactions"].append(reaction_dict)

            # Convert set to list (set -> list == unique entries only)
            path_dict["allKOnumbers"] = list(path_dict["allKOnumbers"])
            
            # Add pathway dictionary to category dictionary
            cat_dict["pathways"].append(path_dict)
            
        # Now that we have the populated dictionaries, remove the list of IDs
        del cat_dict["pathwayIDs"]
            
    # Print progress
    print("Step 3 complete.")
    
    # Return current data structure
    return org_dict


<br><br><br><br><br><br><br><br><br>


<a id='Step4'></a>
## <span style="margin:auto;display:table">Step 4: Determing Seeds/Nonseeds and Children/Parents</span>

#### Required input: 
> Output from Step 3 <br>
> ```max_component_size``` &emsp; *(default = 5)*<br>
> ```distance_transformation``` &emsp; *(default = "1/x")*<br>
> ```weight_measure``` &emsp; *(default = "Degree Centrality")*<br>
                        

#### Overview of process:

As mentioned before, this pipeline is specifically designed to be run in conjunction with the  **NetInteract** tool, which is inspired by the Borenstein lab's NetCooperate tool <sup>4,5</sup>. 

##### SEEDS vs. NON-SEEDS:

The original NetCooperate tool requires seed information to compute complementarity and support scores. <span style="color:red;">"Seeds"</span> are compounds that are exogenously acquired, while <span style="color:blue;">"non-seeds"</span> are endogenously acquired. In simpler terms, this means that <span style="color:red;">seeds</span> are the outside input into a metabolic network, while <span style="color:blue;">non-seeds</span> are the inside capacity of that network. An organism requires <span style="color:red;">seeds</span> in order to produce <span style="color:blue;">non-seeds</span>. (See the below figure for a visual representation.)

<img src="./Images/Step 4 - Seeds and Nonseeds.svg" style="width: 60%;">

In order to distinguish <span style="color:red;">seeds</span> from <span style="color:blue;">non-seeds</span>, we use network topology. (<i>Theory borrowed from the Borenstein lab's methods<sup>6</sup>. Code inspired by the tool PhyloMint <sup>7,8</sup></i>.) 
    
Here is a simplifed explanation of the process:
> **1)** The nested data structure is first converted into edge-notation and then into a directed graph. <br><br>
> **2)** Then, the graph's strongly connected components (SCCs) are determined using Networkx's implementation of Kosaraju's algorithm<sup>9</sup>. <br><br>
> **3)** SCCs that do NOT have any incoming edges are classified as <span style="color:red;">seed groups</span>, such that all compounds inside of the SCC have an equal chance of being a true <span style="color:red;">seed</span>. Compounds within SCCs with at least one incoming edge are classified as <span style="color:blue;">non-seeds</span>.<br>
   
Using this method, we can distinguish whether each compound is a <span style="color:red;">seed</span> or <span style="color:blue;">non-seed</span>. <br>

Additionally, for each <span style="color:blue;">non-seed</span>, we find its “weight” using degree centrality. This is a common metric for finding the importance of nodes in a network and is defined as the number of incoming and outgoing reactions from that node. <br>

##### PARENT / CHILD RELATIONSHIPS:
   
Since you can think of <span style="color:blue;">non-seeds</span> as being in the center of the network and the <span style="color:red;">seeds</span> as being on the outside, the parent <span style="color:red;">seed groups</span> of a <span style="color:blue;">non-seed</span> are the ones that eventually trickle down the network to that <span style="color:blue;">non-seed</span> (hence, “parent”). One <span style="color:blue;">non-seed</span> can have multiple parent <span style="color:red;">seed groups</span>, and one <span style="color:red;">seed group</span> can be the parent of many <span style="color:blue;">non-seeds</span>, so it’s not a one-to-one comparison. <br>

The original NetCooperate tool does not consider the downstream metabolic effects of each <span style="color:red;">seed</span>, considering them all of equal importance. However, this is not necessarily true. One <span style="color:red;">seed</span> may go on to impact half of the entire network of <span style="color:blue;">non-seeds</span>, while another may only impact a handful of <span style="color:blue;">non-seeds</span>. <br>

To account for this variation in <span style="color:red;">seed</span> impact, we've implemented the additional step of finding relationships between metabolites. For each "parent" <span style="color:red;">seed</span>, we find all of the <span style="color:blue;">non-seeds</span> that are its "children", as well as the distance (in # of reactions) between these compounds. (We theorized a seed group that is directly connected to a nonseed would have more impact than one that is 25 reactions away.) This information will be used in the **NetInteract** tool to calibrate interaction scores based on their predicted downstream metabolic effects.

The final dictionary is populated and returned as output.

---

<b>References / Additional Resources:</b><br>
><sup>4</sup> https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-015-0588-y <br>
><sup>5</sup> http://depts.washington.edu/elbogs/NetCooperate/NetCooperateWeb.cgi <br>
><sup>6</sup> https://www.pnas.org/content/suppl/2008/09/11/0806162105.DCSupplemental/0806162105SI.pdf#nameddest=STXT <br>
><sup>7</sup> https://github.com/mgtools/PhyloMint <br>
><sup>8</sup> https://pubmed.ncbi.nlm.nih.gov/33125363/<br>
><sup>9</sup> https://networkx.org/documentation/networkx-1.9/reference/generated/networkx.algorithms.components.strongly_connected.strongly_connected_components.html
----


In [12]:
"""
Steps:
1) Extract edge-notation file from nested data structure
2) Convert to graph
3) Use Borenstein lab's implementation of the Kosaraju algorithm
    to determine seeds, seedgroups, nonseeds, and allnodes
4) Determine seed "parents" (predecessors) and nonseed "children" (successors)
5) Update the data structure with seed information

Returns: a dictionary in the form 
    {
        "organism": {
            "organismID" :
            "organismName" :
            "organismSource" : 
            "organismType"
            "categories" : [
                {
                "categoryName" :
                "pathways" : [
                    {
                    "pathwayID" :
                    "pathwayName" :
                    "allKOnumbers" :
                    "reactions" : [
                        {
                        "reactionID" :
                        "reactionType" :
                        "KOnumbers" :
                        "edges" :
                        "compounds" : [
                            {
                            "compoundID" :
                            "compoundType" :
                            "compoundName"
                            "isSeed" :
                            "seedGroupID" :
                            }
                            ,...
                        ]
                        }
                        ,...
                    ]
                    }
                    ,...
                ]
                }
                ,...
            ]
            "allNodes" : []
            "allSeeds" : []
            "allSeedGroups" : [
                {
                "seedGroupID":
                "seedCompoundIDs": []
                "nonSeedSuccessors": [
                    {
                    "compoundID" :
                    "distance" :
                    "weight" :
                    }
                    ]
                "networkImpact" : 
                    {
                    "total" : 
                        {
                        "score" :
                        }
                    "unique" : 
                        {
                        "score" :
                        "nonSeedIDs" : []
                        }
                    }
                }
                ,...
            ]
            "allNonSeeds" : [
                {
                "compoundID": 
                "weight":
                "seedGroupPredecessors": [
                    {
                    "seedGroupID" :
                    "distance" :
                    }
                }
            ]
        }
    }


"""

def step4(org_dict, max_component_size = 5, distance_transformation = "1/x", \
                        weight_measure = "Degree Centrality"):

    
    #---------------------------------------------------------
    # Based on a seed and the strongly connected component it's in, 
    # return all nonseed successors
    def find_successors(seed, SCC, graph, distance_transformation, all_node_weights):
        
        # Find the successors from that seed
        successors_temp = nx.dfs_successors(graph, seed)
        # In the dfs_successors object, successors are stored as keys AND values -- grab all
        successors_keys = [k for k in successors_temp.keys() if k not in SCC]
        successors_values = [i for v in successors_temp.values() for i in v if i not in SCC]
        successors_all = list(set(successors_keys + successors_values))

        successors_list = [] # Will append a dictionary for each successor node
        # For each of the successors, find the shortest path between them
        for successor in successors_all:
            shortest_path_length = nx.shortest_path_length(graph, source = seed, target = successor)

            # By scaling this way, successors are weighted less if they are further away from the seed
            if distance_transformation == "1/x":
                scaled_distance = 1/shortest_path_length

            # Find weight of that specific successor
            successor_weight = all_node_weights[successor]

            successor_tempdict = {"compoundID" : successor,\
                                 "distance": scaled_distance,\
                                 "weight" : successor_weight}

            successors_list.append(successor_tempdict)

        return successors_list
        
    #---------------------------------------------------------
    # Inspired by PhyloMint's BuildGraphNetX.py script
    # https://github.com/mgtools/PhyloMint
    
    # Improvements by PhyloMint:
    # Improves upon NetCooperate module implementation which erroneously discards 
    # certain cases of SCCs (where a smaller potential SCC lies within a larger SCC)

    # Improvements here:
    # FIX DESCRIPTION!
    # - Still uses nodes in SCCs > maxComponentSize by breaking them up into SCCs of size 1
    # - Adds in maximum number of successors from seed group (i.e., downstream impact 
    # of that node on the network), as cited in the NetCmpt literature
    # - Also scales these successors based on network centrality measures and distance from seed
    # input (so closer and more important nonseeds are weighted higher)
    
    # Note: Decided to keep all seed information in the allSeedGroups variable 
    # instead of for each seed in the compounds variable.
    
    def calculate_seeds(edges, max_component_size = 5, distance_transformation = "1/x", \
                        weight_measure = "Degree Centrality"):
        
        # Construct directed graph from edge-notation format
        graph = nx.parse_edgelist(edges, delimiter='\t', create_using = nx.DiGraph())

        # Get strongly connected components (SCC)
        SCCs = nx.strongly_connected_components(graph)
        
        # Grab weight scores for entire network
        if weight_measure == "Degree Centrality":
            all_node_weights = nx.degree_centrality(graph)

        #------------INTIALIZE SOME VARIABLES------------
        
        # Initialize all_nodes, all_nonseeds, and all_seeds
        all_nodes = list(graph.nodes()) # List of compound IDs
        all_seeds = [] # Will become list of seed compounds IDs
        
        # Initialize all_seedgroups
        # Will become a list of dictionaries in the form:
        #    [
        #        {
        #        "seedGroupID":
        #        "seedCompoundIDs": []
        #        "nonSeedSuccessors": [
        #            {
        #            "compoundID" :
        #            "distance" :
        #            "weight" :
        #            }
        #        ]
        #        "networkImpactTotal":
        #        "networkImpactUnique": 
        #        }
        #        ,...
        #    ]
        all_seedgroups = []
        seedgroup_ID = 0 # Will be incremented
        
        
        #------------ITERATE OVER SCCs IN NETWORK TO FIND SEEDGROUPS------------
        # Note: First we find seedgroups, then we find nonseeds, then we find seedgroup impact.
        # We have to do this in stages because if we start finding the nonseed predecessor seed groups before
        # all seed groups have been found, we'll have incomplete results.
        
        # For each SCC in the network
        for SCC in SCCs:
            
            # Convert to list
            SCC_lst = list(SCC)
            
            #------------IF SCC SIZE > Max SIZE------------
            
            # Filter out SCC larger than threshold
            # Check each individual node for seed/nonseed distinction
            if len(SCC_lst) > max_component_size:
                
                for node in SCC_lst:
                    
                    # This probably won't happen though at the node level
                    # If there are no incoming edges:
                    if graph.in_degree(node) == 0:
                        
                        # Add to all_seeds list
                        all_seeds.append(node)

                        successors_list = find_successors(seed = node, SCC = list(node), graph = graph,\
                                                          distance_transformation = distance_transformation, \
                                                          all_node_weights = all_node_weights)

                        # Make a dictionary for this seed group
                        seedgroup = {"seedGroupID": seedgroup_ID,\
                                     "seedCompoundIDs": list(node),\
                                     "nonSeedSuccessors": successors_list}

                        # Add it to the list of seed groups
                        all_seedgroups.append(seedgroup)
                        seedgroup_ID += 1
                        
            #------------IF SCC SIZE == 1------------
            
            # If the SCC only has one node
            elif len(SCC_lst) == 1:
                
                # Check that there are no incoming edges
                if graph.in_degree(SCC_lst[0]) == 0:
                    
                    # Add to all_seeds list
                    all_seeds.append(SCC_lst[0])

                    successors_list = find_successors(seed = SCC_lst[0], SCC = SCC_lst, graph = graph,\
                                                      distance_transformation = distance_transformation,\
                                                      all_node_weights = all_node_weights)

                    # Make a dictionary for this seed group
                    seedgroup = {"seedGroupID": seedgroup_ID,\
                                 "seedCompoundIDs": SCC_lst,\
                                 "nonSeedSuccessors": successors_list}
                    
                    # Add it to the list of seed groups
                    all_seedgroups.append(seedgroup)
                    seedgroup_ID += 1
                    
                   
            #------------IF 1 < SCC SIZE < MAX SIZE------------
            
            # If the SCC has between 2 and the maximum threshold of nodes
            else:
                
                # Check that SCC is self-contained (i.e., all in-edges are in the SCC)
                SCC_inedges = [edge[0] for node in SCC_lst \
                              for edge in graph.in_edges(node) \
                              if edge[0] not in SCC_lst]
                
                # Check that SCC_inedges is empty (== the SCC is self-contained)
                if not SCC_inedges:
                    
                    # Initialize SCC_successors (will be updated after going through seeds)
                    SCC_successors = []
        
                    for seed in SCC_lst:
                
                        # Add to seeds dictionary
                        all_seeds.append(seed)
                        
                        successors_list = find_successors(seed = seed, SCC = SCC_lst, graph = graph,\
                                                          distance_transformation = distance_transformation, \
                                                          all_node_weights = all_node_weights)

                        for successor_dict in successors_list:
                            
                            successor_location = next((i for i, j in enumerate(SCC_successors)\
                                                       if successor_dict["compoundID"] == j["compoundID"]), None)
                            
                            # If that nonseed successor is NOT already in SCC_successors:
                            if successor_location is None:
                                SCC_successors.append(successor_dict)
                             
                            # Else if it's in SCC_successors, update scaled_distance if it's greater this time 
                            # (i.e., if the original distance was smaller)
                            else:
                                previous_distance = SCC_successors[successor_location]["distance"]
                                if successor_dict["distance"] > previous_distance:
                                    SCC_successors[successor_location]["distance"] = successor_dict["distance"]
                                
                    # Make a dictionary for this seed group
                    seedgroup = {"seedGroupID": seedgroup_ID,\
                                 "seedCompoundIDs": SCC_lst,\
                                 "nonSeedSuccessors": SCC_successors}
                    
                    # Add it to the list of seed groups
                    all_seedgroups.append(seedgroup)
                    seedgroup_ID += 1
                    
                    
        #------------FIND ALL_NONSEEDS AND THEIR "PARENT" SEED GROUPS------------
        
        # Initialize all_nonseeds
        # Will become a list of dictionaries in the form:
        #    [
        #        {
        #        "compoundID": 
        #        "weight":
        #        "seedGroupPredecessors": 
        #            [
        #                {
        #                "seedGroupID" :
        #                "distance" :
        #                }
        #            ]
        #        }
        #    ]
        all_nonseeds = []
        
        # Grab nonseed IDs
        all_nonseed_IDs = list(set(all_nodes) - set(all_seeds))
        number_of_nonseeds = len(all_nonseed_IDs)
        
        # Will hold any times there's only one seed group that has that nonseed as a successor
        #      {"seedGroupID": ["successorCompoundID", ....]}
        uniqueimpacts_of_seedgroup_predecessors = {}
        
        #------------
        # Iterate over all nonseed IDs
        for nonseed_ID in all_nonseed_IDs:
            
            # Find weight of that specific nonseed
            nonseed_weight = all_node_weights[nonseed_ID]
            
            #------------
            # Find seed groups predecessors (i.e., which seed groups is this nonseed a successor of?)
            seedgroup_predecessors = []
            
            for seedgroup in all_seedgroups:
                
                # Check if this nonseed is one of its successors
                successor_location = next((i for i, j in enumerate(seedgroup["nonSeedSuccessors"])\
                                           if j["compoundID"] == nonseed_ID), None)
                # If it is
                if successor_location is not None:
                    seedgroup_predecessor = {"seedGroupID" : seedgroup["seedGroupID"],\
                                             "seedCompoundIDs" : seedgroup["seedCompoundIDs"],\
                                             "distance" : seedgroup["nonSeedSuccessors"][successor_location]["distance"]}
                
                    seedgroup_predecessors.append(seedgroup_predecessor)
                
            #------------
            # If there's only one, grab as unique contribution
            if len(seedgroup_predecessors)==1:
                
                seedgroup_predecessor_ID = seedgroup_predecessors[0]["seedGroupID"]
                
                # Check if it's already in that dictionary
                if seedgroup_predecessor_ID not in uniqueimpacts_of_seedgroup_predecessors.keys():
                    uniqueimpacts_of_seedgroup_predecessors[seedgroup_predecessor_ID] = [nonseed_ID]
                    
                else:
                    uniqueimpacts_of_seedgroup_predecessors[seedgroup_predecessor_ID].append(nonseed_ID)
            
            #------------
            # Check for errors
            elif len(seedgroup_predecessors)==0:
                print("Error: no seed group predecessors found for nonseed: ", str(nonseed_ID))
                    
            #------------
            # Make a dictionary for this nonseed
            nonseed = {"compoundID": nonseed_ID,\
                       "weight": nonseed_weight,\
                       "seedGroupPredecessors": seedgroup_predecessors}
                       
            # Add it to the list of nonseeds
            all_nonseeds.append(nonseed)
                   
        #------------FIND IMPACT SCORE FOR EACH "PARENT" SEED GROUP------------
        # Seems confusing, but the easiest way was to compute the non-seeds and their parents first
        
        for seedgroup in all_seedgroups:
            
            seedgroup_ID = seedgroup["seedGroupID"]
            unique_impact = 0.0
            total_impact = 0.0
                       
            # Calculate the unique impact of this seed group
            if seedgroup_ID in uniqueimpacts_of_seedgroup_predecessors.keys():
                unique_nonseeds = uniqueimpacts_of_seedgroup_predecessors[seedgroup_ID]
                
                for nonseed_ID in unique_nonseeds:
                    nonseed_location = next((i for i, j in enumerate(seedgroup["nonSeedSuccessors"])\
                                           if j["compoundID"] == nonseed_ID), None)
                    nonseed_distance = seedgroup["nonSeedSuccessors"][nonseed_location]["distance"]
                    nonseed_weight = seedgroup["nonSeedSuccessors"][nonseed_location]["weight"]
                    nonseed_impact = (nonseed_distance * nonseed_weight) / number_of_nonseeds
                    unique_impact += nonseed_impact
                       
            else:
                unique_nonseeds = None
                    
            # Calculate the total impact of this seed group
            for successor_dict in seedgroup["nonSeedSuccessors"]:
                nonseed_impact = (successor_dict["distance"] * successor_dict["weight"]) / number_of_nonseeds
                total_impact += nonseed_impact
                       
            seedgroup["networkImpact"] = {}
            seedgroup["networkImpact"]["total"] = {"score": total_impact}
            seedgroup["networkImpact"]["unique"] = {"score": unique_impact,\
                                                    "nonSeedIDs": unique_nonseeds}
            
                       
        #------------RETURN EVERYTHING------------
        
        #print(len(nodes), len(non_seeds), len(seeds))
        
        # all_seeds = List of compound IDs
        # all_seedgroups = List of dictionaries, one for each seed group
        # all_nonseeds = List of dictionaries, one for each nonseed
        # all_nodes = List of compound IDs
        return([all_seeds, all_seedgroups, all_nonseeds, all_nodes])

      
    #---------------------------------------------------------

    def update_with_seeds(org_dict, all_seeds, all_seedgroups, all_nonseeds, all_nodes):

        for cat_dict in org_dict["organism"]["categories"]:
            for path_dict in cat_dict["pathways"]:
                for reaction_dict in path_dict["reactions"]:
                    for compound_dict in reaction_dict["compounds"]:
                        
                        compound_ID = compound_dict["compoundID"]
                        all_nonseed_IDs = [n["compoundID"] for n in all_nonseeds]
                        
                        if compound_ID in all_seeds:
                            compound_dict["isSeed"] = True

                            # Find which seedgroup it's in
                            seed_group_location = next((i for i, j in enumerate(all_seedgroups)\
                                                        if compound_ID in j["seedCompoundIDs"]), None)
                            
                            compound_dict["seedGroupID"] = all_seedgroups[seed_group_location]["seedGroupID"]

                        elif compound_ID in all_nonseed_IDs:
                            compound_dict["isSeed"] = False
                            compound_dict["seedGroupID"] = None

                        else:
                            print("Error, not in seeds or nonseeds", compound_ID)

        org_dict["organism"]["allNodes"] = all_nodes
        org_dict["organism"]["allSeeds"] = all_seeds
        org_dict["organism"]["allSeedGroups"] = all_seedgroups
        org_dict["organism"]["allNonSeeds"] = all_nonseeds

        return org_dict

    #---------------------------------------------------------
    # Extract edge-notation file
    edges = extract_TSV(org_dict = org_dict)
    
    # Calculate the seed sets
    all_seeds, all_seedgroups, all_nonseeds, all_nodes = calculate_seeds(edges)
    
    # Update the data structure with seed information
    org_dict = update_with_seeds(org_dict = org_dict, \
                                 all_seeds = all_seeds, all_seedgroups = all_seedgroups,\
                                 all_nonseeds= all_nonseeds, all_nodes = all_nodes)
    
    # Print progress
    print("Step 4 complete.")
    
    # Return
    return org_dict


<br><br><br><br><br><br><br><br><br>

<a id='Step5'></a>
## <span style="margin:auto;display:table">Step 5: Send to Files</span>


#### Required input:
> Output from Step 4 <br>
> ```genome_input``` <br>
> ```all_compound_names```

#### Overview of process:

The final data structure from Step 4 is written to several files:
>- A JSON file containing the entire structure. <br> <br>
>- A CSV file showing the hierarchy of category, pathway, reaction, and compound, as well as whether each compound is a <span style="color:red;">seed</span> or a <span style="color:blue;">non-seed</span>. <br> <br>
>- A CSV file showing the relationships between <span style="color:red;">seed"parents"</span> and their <span style="color:blue;">non-seed "children"</span>.


---


In [13]:
"""
Steps:
1) Send full dictionary structure to JSON file
2) Create pandas DataFrame, where each row corresponds to a compound.
   For that compound, show which reaction, pathway, and category it's in,
   as well as whether or not it is a seed/non-seed.
3) Send DataFrame from Step 2 to CSV file.
4) Create pandas DataFrame, where each row corresponds to a unique
   child non-seed and parent seed relationship. 
5) Send DataFrame from Step 4 to CSV file.

Does not return anything.
    
    
"""

def step5(org_dict, genome_input, all_compound_names):
    
    #------------SEND ENTIRE DICTIONARY TO JSON FILE------------
    with open(genome_input["destination_filepath_json"], "w") as f:
        json.dump(org_dict, f)
      
    #------------SEND ENTIRE DICTIONARY TO CSV FILE------------
    org_df = pd.DataFrame()
    org_nonseeds_df = pd.DataFrame()

    organismID = org_dict["organism"]["organismID"]
    organismName = org_dict["organism"]["organismName"]
    organismSource = org_dict["organism"]["organismSource"]

    #------------
    #Category - Pathway - Reaction - Compound
    for cat_dict in org_dict["organism"]["categories"]:

        categoryName = cat_dict["categoryName"]

        for path_dict in cat_dict["pathways"]:

            pathwayID = path_dict["pathwayID"]
            pathwayName = path_dict["pathwayName"]

            for reaction_dict in path_dict["reactions"]:

                reactionID = reaction_dict["reactionID"]
                reactionType = reaction_dict["reactionType"]
                KOnumbers = ','.join(reaction_dict["KOnumbers"])
                edges = ','.join(reaction_dict["edges"])

                for compound_dict in reaction_dict["compounds"]:
                    compoundID = compound_dict["compoundID"]
                    compoundName = compound_dict["compoundName"]
                    compoundType = compound_dict["compoundType"]
                    isSeed = compound_dict["isSeed"]
                    seedGroupID = compound_dict["seedGroupID"]

                    org_df_row = {'Organism ID': organismID,\
                                     'Organism Name': organismName,\
                                     'Organism Source': organismSource,\
                                     'Category Name': categoryName,\
                                     'Pathway ID' : pathwayID,\
                                     'Pathway Name' : pathwayName,\
                                     'Reaction ID' : reactionID,\
                                     'Reaction Type' : reactionType,\
                                     'Reaction KO Numbers' : KOnumbers,\
                                     'Reaction Edge-Notation' : edges,\
                                     'Compound ID' : compoundID,\
                                     'Compound Name' : compoundName,\
                                     'Compound Type' : compoundType,\
                                     'Is Compound Seed?' : isSeed,\
                                     'Compound Seed Group' : seedGroupID}

                    org_df = org_df.append(org_df_row, ignore_index = True)

    org_df.to_csv(genome_input["destination_filepath_csv"], index=False,\
                  columns = ['Organism Name','Organism Source','Organism ID',\
                             'Category Name','Pathway Name','Pathway ID',\
                             'Reaction ID','Reaction Type','Reaction KO Numbers',\
                             'Reaction Edge-Notation','Compound ID','Compound Name',\
                             'Compound Type','Is Compound Seed?','Compound Seed Group'])
           
    #------------
    #Child Nonseed and Parent Seeds Relationships   
    for nonseed_dict in org_dict["organism"]["allNonSeeds"]:
        nonseed_compound_ID = nonseed_dict["compoundID"]
        nonseed_weight = nonseed_dict["weight"]
        nonseed_compound_name = all_compound_names[nonseed_compound_ID]
        
        for seed_group_predecessor in nonseed_dict["seedGroupPredecessors"]:
            seed_group_ID = seed_group_predecessor["seedGroupID"]
            seed_group_distance = seed_group_predecessor["distance"]
            
            for seed_predecessor in seed_group_predecessor["seedCompoundIDs"]:
                seed_predecessor_compound_name = all_compound_names[seed_predecessor]
                
                org_nonseeds_df_row = {'Organism ID': organismID,\
                                       'Organism Name': organismName,\
                                       'Organism Source': organismSource,\
                                       "Nonseed Compound ID": nonseed_compound_ID,\
                                       "Nonseed Compound Name": nonseed_compound_name,\
                                       "Nonseed Weight": nonseed_weight,\
                                       "Seed Group Predecessor - ID": seed_group_ID,\
                                       "Seed Group Predecessor - Distance": seed_group_distance,\
                                       "Seed Group Predecessor - Seed Compound ID": seed_predecessor,\
                                       "Seed Group Predecessor - Seed Compound Name": seed_predecessor_compound_name}
            
                org_nonseeds_df = org_nonseeds_df.append(org_nonseeds_df_row, ignore_index = True)
        
    org_nonseeds_df.to_csv(genome_input["destination_filepath_csv_nonseeds"], index=False,\
                  columns = ['Organism Name',\
                             'Organism Source',\
                             'Organism ID',\
                             "Nonseed Compound ID",\
                             "Nonseed Weight",\
                             "Seed Group Predecessor - ID",\
                             "Seed Group Predecessor - Distance",\
                             "Seed Group Predecessor - Seed Compound ID"])

    # Print progress
    print("Step 5 complete.")
    

<br><br><br><br><br><br><br><br><br>

<a id='Step6'></a>
## <span style="margin:auto;display:table;">Step 6: Visualize Network</span>

#### Required input:
> ```genome_inputs``` <br>

#### Overview of process:




---


In [14]:
"""
Steps:
1) Extract tab-delimited edges (i.e., reactions) from network.
2) Define colors for each node (i.e., compound), so that seeds are red and non-seeds are blue.
3) Draw network using networkx.
4) Save network to PDF file.

Does not return anything.
    
    
"""

def step6(org_dict, genome_input):
    
    # Extract tab-delimited edges (i.e., reactions) from network
    data = extract_TSV(org_dict = org_dict)
        
    # Grab all compounds and seeds
    allnodes = set(org_dict["organism"]["allNodes"])
    seeds = set(org_dict["organism"]["allSeeds"])
    
    # Initialize map that will determine colors for each compound
    color_map = {}

    # Color all seeds red
    for s in seeds:
        color_map[s] = 'red'
    # Color all non-seeds blue
    for s in [n for n in allnodes if n not in seeds]:
        color_map[s] = 'blue'
                
    # Initialize networkx graph using extracted TSV
    G = nx.parse_edgelist(data, delimiter='\t', create_using = nx.DiGraph())
    
    # Find a color for each compound (i.e., node) based on the color_map
    values = [color_map.get(node) for node in G.nodes()]
    
    # Intialize figure
    fig = plt.figure()
    
    # Draw network
    nx.draw_kamada_kawai(G, node_size = 10, arrowsize = 0.4, alpha = 0.2, node_color=values)
    
    # Save figure to file
    plt.savefig(genome_input["destination_filepath_pdf"], bbox_inches='tight')
    
    # Close figure so that it won't pop up after completion
    plt.close(fig)
    
    # Print progress
    print("Step 6 complete.")
    

<br><br><br><br><br><br><br><br><br>

<a id='Step7'></a>
## <span style="margin:auto;display:table;">Step 7: Summary Statistics and Network Comparisons</span>

#### Required input:
> ```all_inputs``` <br>

#### Overview of process:




---


In [15]:
"""
Steps:
1) Find all genomes that have completed metabolic networks
2) For each completed network, extract summary statistics 
   (i.e., # reactions, # compounds, # seeds, % compounds that are seeds),
   as well as sets of reactions, compounds, and seeds to be used in pair-wise comparisons
3) Print summary statistics to CSV file.
3) Get all unique pairs of completed networks
4) For each pair of networks, find overlaps, differences, and an overall Jaccard similarity score
   in terms of their reactions, compounds, and seeds.
5) Print pair-wise comparisons to CSV file.

Does not return anything.
    
    
"""

def step7(all_inputs):
    
    #-------------------------    
    # Find all genomes that have completed metabolic networks
    complete_inputs = [i for i in all_inputs if i["partB_network_complete"] == True]
    
    
    # Make Summary Statistics Directory
    try:
        os.makedirs("./Summary Statistics")
    # Don't make anything if the directory already exists
    except FileExistsError:
        pass
                    
    # Create empty pandas DataFrame to hold summary statistics 
    # (i.e.,  # reactions, # seeds, # compounds, % seeds)
    summary_stats_df = pd.DataFrame()
    
    # Iterate over completed networks
    for complete_input in complete_inputs:
        
        # Open network file
        with open(complete_input["destination_filepath_json"], "r") as f:
            
            # Load network
            network = json.load(f)

            # Extract all reactions, seeds, and compounds
            # (to be used in pair-wise comparisons)
            reaction_set = set(extract_TSV(org_dict = network))
            seed_set = set(network["organism"]["allSeeds"])
            compound_set = set(network["organism"]["allNodes"])

            # Add sets to complete_input dictionary
            complete_input["reaction_set"] = reaction_set
            complete_input["seed_set"] = seed_set
            complete_input["compound_set"] = compound_set
            
            # Extract summary statistics for this network
            summary_stats_row = {"Organism Name" : complete_input["organism_name"],\
                                 "Organism ID" : complete_input["organism_ID"],\
                                 "Organism Source" : complete_input["database"],\
                                 "# of Reactions" : len(reaction_set),\
                                 "# of Seeds" : len(seed_set),\
                                 "# of Compounds" : len(compound_set),\
                                 "% of Compounds that are Seeds" : len(seed_set) / len(compound_set)}
            
            # Append to DataFrame
            summary_stats_df = summary_stats_df.append(summary_stats_row, ignore_index = True)
        
    # Print summary statistics DataFrame to file
    summary_stats_df.to_csv("./Summary Statistics/Summary Statistics.csv", index = False,\
                           columns = ["Organism Name", "Organism ID" ,"Organism Source",\
                                      "# of Reactions", "# of Seeds", "# of Compounds", \
                                      "% of Compounds that are Seeds"])
    
    #-------------------------    
    # Create empty pandas DataFrame to hold pair-wise comparisons
    comparison_df = pd.DataFrame()
    
    # Get all unique pairs of completed networks
    combos = itertools.combinations(complete_inputs, 2)
    
    # Iterate over pairs
    for combo in combos:
        
        # Find differences, overlap, and Jaccard similarity in terms of reactions
        reactions_only_in_A = len(combo[0]["reaction_set"] - combo[1]["reaction_set"])
        reactions_only_in_B = len(combo[1]["reaction_set"] - combo[0]["reaction_set"])
        reactions_in_both = len(combo[0]["reaction_set"] & combo[1]["reaction_set"])
        reactions_in_either = len(combo[0]["reaction_set"] | combo[1]["reaction_set"])
        reactions_jaccard_similarity = reactions_in_both / reactions_in_either
        
        # Find differences, overlap, and Jaccard similarity in terms of compounds
        compounds_only_in_A = len(combo[0]["compound_set"] - combo[1]["compound_set"])
        compounds_only_in_B = len(combo[1]["compound_set"] - combo[0]["compound_set"])
        compounds_in_both = len(combo[0]["compound_set"] & combo[1]["compound_set"])
        compounds_in_either = len(combo[0]["compound_set"] | combo[1]["compound_set"])
        compounds_jaccard_similarity = compounds_in_both / compounds_in_either
        
        # Find differences, overlap, and Jaccard similarity in terms of seeds
        seeds_only_in_A = len(combo[0]["seed_set"] - combo[1]["seed_set"])
        seeds_only_in_B = len(combo[1]["seed_set"] - combo[0]["seed_set"])
        seeds_in_both = len(combo[0]["seed_set"] & combo[1]["seed_set"])
        seeds_in_either = len(combo[0]["seed_set"] | combo[1]["seed_set"])
        seeds_jaccard_similarity = seeds_in_both / seeds_in_either
        
        # Compile DataFrame row
        comparison_df_row1 = {"Network A - Organism Name" : combo[0]["organism_name"],\
                             "Network A - Organism ID" : combo[0]["organism_ID"],\
                             "Network A - Organism Source" : combo[0]["database"],\
                             "Network B - Organism Name" : combo[1]["organism_name"],\
                             "Network B - Organism ID" : combo[1]["organism_ID"],\
                             "Network B - Organism Source" : combo[1]["database"],\
                             "Reactions only in Network A": reactions_only_in_A ,\
                             "Reactions only in Network B": reactions_only_in_B,\
                             "Reactions in both": reactions_in_both,\
                             "Jaccard similarity of reactions": reactions_jaccard_similarity,\
                             "Compounds only in Network A": compounds_only_in_A ,\
                             "Compounds only in Network B": compounds_only_in_B,\
                             "Compounds in both": compounds_in_both,\
                             "Jaccard similarity of compounds": compounds_jaccard_similarity,\
                             "Seeds only in Network A": seeds_only_in_A ,\
                             "Seeds only in Network B": seeds_only_in_B,\
                             "Seeds in both": seeds_in_both,\
                             "Jaccard similarity of seeds": seeds_jaccard_similarity}
        
        # Compile DataFrame row (networks flipped)
        comparison_df_row2 = {"Network A - Organism Name" : combo[1]["organism_name"],\
                             "Network A - Organism ID" : combo[1]["organism_ID"],\
                             "Network A - Organism Source" : combo[1]["database"],\
                             "Network B - Organism Name" : combo[0]["organism_name"],\
                             "Network B - Organism ID" : combo[0]["organism_ID"],\
                             "Network B - Organism Source" : combo[0]["database"],\
                             "Reactions only in Network A": reactions_only_in_B ,\
                             "Reactions only in Network B": reactions_only_in_A,\
                             "Reactions in both": reactions_in_both,\
                             "Jaccard similarity of reactions": reactions_jaccard_similarity,\
                             "Compounds only in Network A": compounds_only_in_B,\
                             "Compounds only in Network B": compounds_only_in_A,\
                             "Compounds in both": compounds_in_both,\
                             "Jaccard similarity of compounds": compounds_jaccard_similarity,\
                             "Seeds only in Network A": seeds_only_in_B ,\
                             "Seeds only in Network B": seeds_only_in_A,\
                             "Seeds in both": seeds_in_both,\
                             "Jaccard similarity of seeds": seeds_jaccard_similarity}
        
        # Append to DataFrame
        comparison_df = comparison_df.append(comparison_df_row1, ignore_index = True)
        comparison_df = comparison_df.append(comparison_df_row2, ignore_index = True)
        
    # Print pair-wise comparisons DataFrame to file
    comparison_df.to_csv("./Summary Statistics/Comparisons.csv", index = False,\
                        columns = ["Network A - Organism Name",\
                                   "Network A - Organism ID",\
                                   "Network A - Organism Source",\
                                   "Network B - Organism Name",\
                                   "Network B - Organism ID",\
                                   "Network B - Organism Source",\
                                   "Reactions only in Network A",\
                                   "Reactions only in Network B",\
                                   "Reactions in both",\
                                   "Jaccard similarity of reactions",\
                                   "Compounds only in Network A",\
                                   "Compounds only in Network B",\
                                   "Compounds in both",\
                                   "Jaccard similarity of compounds",\
                                   "Seeds only in Network A",\
                                   "Seeds only in Network B",\
                                   "Seeds in both",\
                                   "Jaccard similarity of seeds"])
    #-------------------------
    # Make heatmaps
    comparison_df["Network A Label"] = comparison_df["Network A - Organism Name"] + " - " + comparison_df["Network A - Organism Source"] + " - " + comparison_df["Network A - Organism ID"]
    comparison_df["Network B Label"] = comparison_df["Network B - Organism Name"] + " - " + comparison_df["Network B - Organism Source"] + " - " + comparison_df["Network B - Organism ID"]

    heatmap_df_seeds = comparison_df[["Network A Label", "Network B Label", "Jaccard similarity of seeds"]]
    heatmap_df_reactions = comparison_df[["Network A Label", "Network B Label", "Jaccard similarity of reactions"]]
    heatmap_df_compounds = comparison_df[["Network A Label", "Network B Label", "Jaccard similarity of compounds"]]

    # Pivot DataFrames
    heatmap_pivotdf_seeds = heatmap_df_seeds.pivot_table(index='Network A Label', columns='Network B Label', \
                                                         values="Jaccard similarity of seeds")
    heatmap_pivotdf_reactions = heatmap_df_reactions.pivot_table(index='Network A Label', columns='Network B Label', \
                                                         values="Jaccard similarity of reactions")
    heatmap_pivotdf_compounds = heatmap_df_compounds.pivot_table(index='Network A Label', columns='Network B Label', \
                                                         values="Jaccard similarity of compounds")

    df_title_dict = {"\nJaccard Similarity of Seeds\n" :heatmap_pivotdf_seeds,\
                     "\nJaccard Similarity of Reactions\n" :heatmap_pivotdf_reactions,\
                     "\nJaccard Similarity of Compounds\n" :heatmap_pivotdf_compounds}

    for title,df in df_title_dict.items():

        filepath = "./Summary Statistics/" + title.strip()

        fig, ax = plt.subplots(figsize=(40,40))
        ax.set_title(title, fontsize=30)
        ax.set_xlabel("", fontsize=30)
        ax.set_ylabel("", fontsize=30)
        ax.tick_params(axis='both', which='major', labelsize=18)
        sns.heatmap(df, ax = ax, cmap='coolwarm')
        plt.tight_layout()
        # Save figure to file
        plt.savefig(filepath, bbox_inches='tight')
        plt.close(fig)
    
    #-------------------------
    # Print progress
    print("Step 7 complete.")
    

<br><br><br><br><br><br><br><br><br>

<a id='Full_Pipeline_PartB'></a>
## <span style="margin:auto;display:table">Full Pipeline for Part B</span>


In [16]:
"""
Steps:
1) Use the KEGG API to find all compound names
2) Iterate over genome inputs; if part B isn't already complete, run steps 3-6.
3) Run step 7 to compute summary statistics for each network 
   as well as pair-wise comparisons between networks.

Does not return anything.
    
    
"""

def full_pipeline_partB(all_inputs,\
                        max_component_size = 5, \
                        distance_transformation = "1/x", \
                        weight_measure = "Degree Centrality"):
    
    #------------GET DICTIONARY OF COMPOUND NAMES {ID:Name}------------

    all_compound_names = {}
    compounds_str = REST.kegg_list("compound").read()
    glycans_str = REST.kegg_list("glycan").read()

    for line in compounds_str.rstrip().split("\n"):
        compound_ID, compound_name = line.split("\t")
        all_compound_names[compound_ID[4:]] = compound_name

    for line in glycans_str.rstrip().split("\n"):
        glycan_ID, glycan_name = line.split("\t")
        all_compound_names[glycan_ID[3:]] = glycan_name

    #-------------------------   
    # Iterate over inputs (one per network)
    for genome_input in all_inputs:
        
        # Print which genome is currently in progress
        print(genome_input["organism_name"], genome_input["database"], genome_input["organism_ID"])
        
        # Check that the network isn't already complete
        if genome_input["partB_network_complete"] == True:
            print("Part B already complete for this genome.")
        
        else:
            step3_output = step3(genome_input = genome_input, \
                                 all_compound_names = all_compound_names)
            step4_output = step4(org_dict = step3_output)
            step5(org_dict = step4_output, genome_input = genome_input,\
                 all_compound_names = all_compound_names)
            step6(org_dict = step4_output, genome_input = genome_input)
            
            # Set this to True, so that this input is included in step 7
            genome_input["partB_network_complete"] = True
            
    #-------------------------   
    # Find summary statistics and comparison metrics and print to file
    step7(all_inputs = all_inputs)
    
    #-------------------------   
    # Print progress
    print("Part B of Full Pipeline Complete")
    

In [17]:
%time full_pipeline_partB(all_inputs = all_inputs)

Acyrthosiphon pisum KEGG api
Part B already complete for this genome.
Buchnera aphidicola Tuc7 NCBI ASM2106v1
Part B already complete for this genome.
Buchnera aphidicola 5A NCBI ASM2108v1
Part B already complete for this genome.
Buchnera aphidicola APS NCBI ASM960v1
Part B already complete for this genome.
Regiella insecticola LSR1 NCBI ASM14362v1
Part B already complete for this genome.
Rickettsiella viridis Ap-RA04 NCBI ASM396675v1
Part B already complete for this genome.
Serratia symbiotica Tuscon NCBI ASM18648v1
Part B already complete for this genome.
Fukatsuia symbiotica 5D NCBI ASM312242v1
Part B already complete for this genome.
Hamiltonella defensa 5AT (AA) NCBI ASM2170v1
Part B already complete for this genome.
Hamiltonella defensa NY26 (AA) NCBI New_ASM277729v1
Part B already complete for this genome.
Hamiltonella defensa MI47 (BB) NCBI New_ASM226940v1
Part B already complete for this genome.
Hamiltonella defensa 5D (BB) NCBI New_ASM312244v1
Part B already complete for this

<br><br><br><br><br><br><br><br><br><br><br><br>

<br><br><br><br><br><br><br><br><br><br>
# <span style="margin:auto;display:table;">Network reconstruction is complete!</span>

## <span style="margin:auto;display:table;">Please open the file titled "NetInteract (Community version).ipynb" to continue.</span>

<br><br><br><br><br>

<br><br><br><br><br><br><br><br><br><br><br><br>